In [ ]:
#calculate distance from robot to object using two images shot in two different direction
'''
VGG_KR_FROM_P Extract K, R from camera matrix.

[K,R,C] = VGG_KR_FROM_P(P [,noscale]) finds K, R, C such that P = K*R*[eye(3) -C].
It is det(R)==1.
K is scaled so that K(3,3)==1 and K(1,1)>0. Optional parameter noscale prevents this.

Works also generally for any P of size N-by-(N+1).
Works also for P of size N-by-N, then C is not computed.
'''


def vgg_rq(S):
    S = S.T
    [Q, U] = np.linalg.qr(S[::-1, ::-1], mode='complete')

    Q = Q.T
    Q = Q[::-1, ::-1]
    U = U.T
    U = U[::-1, ::-1]
    if np.linalg.det(Q) < 0:
        U[:, 0] = -U[:, 0]
        Q[0, :] = -Q[0, :]
    return U, Q


def vgg_KR_from_P(P, noscale=True):
    """
    Extract K, R, C From P
    P = K[R | t] = K [R | -RC]
    """
    N = P.shape[0]
    H = P[:, 0:N]
    print(N, '|', H)
    [K, R] = vgg_rq(H)
    if noscale:
        K = K / K[N-1, N-1]
        if K[0, 0] < 0:
            D = np.diag([-1, -1, np.ones([1, N-2])])
            K = K @ D
            R = D @ R

            test = K*R
            assert (test/test[0, 0] - H/H[0, 0]).all() <= 1e-07

    C = np.linalg.inv(-P[:, 0:N]) @ P[:, -1]
    return K, R, C